Training and Predicting
===========================================================

#### Forked From Kaggle - Andy Harless - XGBoost, LightGBM, OLS and NN
Public Score
0.0643646

1. Global Hyper Parameters
---------------------------------------

In [1]:
FUDGE_FACTOR = 1.1200  # Multiply forecasts by this

XGB_WEIGHT = 0.6200
BASELINE_WEIGHT = 0.0100
OLS_WEIGHT = 0.0620
NN_WEIGHT = 0.0800

XGB1_WEIGHT = 0.8000  # Weight of first in combination of two XGB models

BASELINE_PRED = 0.0115   # Baseline based on mean of training data, per Oleg

2. Import Packages
-----------------------------

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,Normalizer
import lightgbm as lgb
import gc
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,make_scorer
from sklearn.model_selection import cross_val_score,KFold
import random
import datetime as dt
from collections import OrderedDict
import copy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.layers.noise import GaussianDropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer

Using Theano backend.
Using cuDNN version 5110 on context None
Preallocating 9011/11264 Mb (0.800000) on cuda0
Mapped name None to device cuda0: GeForce GTX 1080 Ti (0000:06:00.0)


3. Preparation
--------------------------------

In [3]:
test_only=True
clip_norm=(-0.4,0.419)
month_train=range(1,7)
month_test=range(7,10)
category_features=['propertyzoningdesc','taxdelinquencyyear','propertycountylandusecode','airconditioningtypeid', 'architecturalstyletypeid', 'buildingclasstypeid', 'buildingqualitytypeid', 'decktypeid', 'fips', 'hashottuborspa', 'heatingorsystemtypeid', 'poolcnt', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'propertylandusetypeid', 'regionidcounty', 'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid', 'numberofstories', 'fireplaceflag', 'taxdelinquencyflag']+['basementsqft', 'finishedfloor1squarefeet', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 'poolsizesum', 'yardbuildingsqft17', 'yardbuildingsqft26']
numerical_features=[ 'yearbuilt', 'assessmentyear','bathroomcnt', 'bedroomcnt', 'calculatedbathnbr', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fullbathcnt', 'lotsizesquarefeet', 'roomcnt', 'unitcnt', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount']
extra_features=['latitude', 'longitude',   'rawcensustractandblock', 'regionidcity', 'regionidzip',  'censustractandblock']
prediction_columns=['201607','201608','201609','201707','201708','201709']
prop_lightgbm_path='Data/prop_clean.csv'
prop_xgboost_path='Data/prop_clean.csv'
prop_ols_path='Data/prop_clean.csv'
prop_nn_path='Data/prop_clean.csv'
train_path='Data/train.csv'
sample_path='Data/bak/sample_submission.csv'

In [4]:
def loader(properties_path,drop_list,outlier_bound=(None,None),parse_date=False,labelencode=False,onehotencode=False):
    print( "\nReading data from disk ...")
    print("   Read properties file ...")
    properties = pd.read_csv(properties_path)
    print("   ...")
    if onehotencode:
        print("   One Hot Encoding ...")
        properties=pd.get_dummies(properties, columns=category_features, drop_first=False)
        print("   ...")
    if labelencode:
        print("   Label Encoding ...")
        for c in category_features:
            le=LabelEncoder()
            le.fit(properties[c])
            properties[c]=le.transform(properties[c])
        print("   ...")
    print("   Read training file ...")
    train = pd.read_csv(train_path, parse_dates=["transactiondate"])
    print("   ...")
    train_properties = train.merge(properties, how='left', on='parcelid')
    if parse_date:
        train_properties["transactiondate_year"] = train_properties["transactiondate"].dt.year
        train_properties["transactiondate_month"] = train_properties["transactiondate"].dt.month
    if outlier_bound[0] is not None:
        train_properties=train_properties[train_properties.logerror > outlier_bound[0]]
    if outlier_bound[1] is not None:
        train_properties=train_properties[train_properties.logerror < outlier_bound[1]]
    train_properties=train_properties[train_properties['transactiondate'].dt.month.isin(month_train)]
    train_drop_list=copy.copy(drop_list)
    train_drop_list_all=copy.copy(train_drop_list)
    for dropitem in train_drop_list_all:
        if dropitem not in train_properties.columns.tolist():
            train_drop_list.remove(dropitem)
    y_train = train_properties['logerror']
    x_train = train_properties.drop(train_drop_list, axis=1)
    print(x_train.shape, y_train.shape)
    print("\nPrepare for prediction ...")
    print("   Read sample file ...")
    sample = pd.read_csv(sample_path)
    sample['parcelid'] = sample['ParcelId']
    print("   ...")
    print("   Merge with property data ...")
    if test_only:
        test_properties = train.merge(properties, how='left', on='parcelid')
        test_properties=test_properties[test_properties['transactiondate'].dt.month.isin(month_test)]
    else:
        test_properties = sample.merge(properties, on='parcelid', how='left')
    parcelid=test_properties['parcelid']
    if parse_date:
        test_properties["transactiondate_year"] = 2016.5
        test_properties["transactiondate_month"] = np.mean(month_test)
    test_drop_list=copy.copy(drop_list)+['ParcelId','201610','201611','201612','201710','201711','201712']
    test_drop_list_all=copy.copy(test_drop_list)
    for dropitem in test_drop_list_all:
        if dropitem not in test_properties.columns.tolist():
            test_drop_list.remove(dropitem)
    x_test=test_properties.drop(test_drop_list, axis=1)
    print(x_test.shape)
    return x_train,y_train,x_test,parcelid

In [5]:
def normalize(x_test):
    ssr=StandardScaler()
    ssr.fit(x_test)
    return ssr

In [6]:
valid = pd.read_csv(train_path, parse_dates=["transactiondate"])
y_true=OrderedDict()
y_mask=[]
y_true = valid.logerror[valid['transactiondate'].dt.month.isin(month_test)]
y_date = valid.transactiondate[valid['transactiondate'].dt.month.isin(month_test)]
for yr in [2016,2017]:
    for mt in month_test:
        y_mask.append(np.bitwise_and(y_date.dt.month==mt,y_date.dt.year==yr))
y_mask=np.asarray(y_mask)
y_mask=y_mask.T.astype('int64')
def validate(y_test):
    return np.mean(np.abs(np.sum(y_test*y_mask,1)-y_true))

4. LightGBM
-------------------------

### Process Data for LightGBM

In [7]:
print( "\nProcessing data for LightGBM ..." )

x_train,y_train,x_test,parcelid=loader(prop_lightgbm_path,['parcelid', 'logerror', 'transactiondate'],labelencode=True)
features=category_features+numerical_features+extra_features
d_train = lgb.Dataset(x_train[features], label=y_train, categorical_feature=category_features,free_raw_data=False)


Processing data for LightGBM ...

Reading data from disk ...
   Read properties file ...
   ...
   Label Encoding ...
   ...
   Read training file ...
   ...
((105121, 57), (105121L,))

Prepare for prediction ...
   Read sample file ...
   ...
   Merge with property data ...
(54225, 57)


### Set LightGBM Hyper Parameters

In [8]:
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['sub_feature'] = 0.345    # feature_fraction (small values => use very different submodels)
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 0
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

np.random.seed(0)
random.seed(0)

### Run LightGBM

In [9]:
print("\nFitting LightGBM model ...")

clf = lgb.train(params, d_train, 430, categorical_feature=category_features)


Fitting LightGBM model ...


### LightGBM Prediction

In [10]:
print("\nStart LightGBM prediction ...")

lgb_pred = clf.predict(x_test)

print( "\nUnadjusted LightGBM predictions:" )
print( pd.DataFrame(lgb_pred).head() )


Start LightGBM prediction ...

Unadjusted LightGBM predictions:
          0
0  0.021860
1  0.022435
2  0.022740
3  0.021015
4  0.022069


In [11]:
lgb_pred.shape

(54225L,)

### Validate For LightGBM

In [12]:
validate(np.clip(np.tile(lgb_pred[:,None],[1,6]),clip_norm[0],clip_norm[1]))

0.06942165970198744

### Clean for LightGBM

In [13]:
del x_train,y_train,x_test,parcelid,d_train
gc.collect()

195

5. XGBoost
-----------------------------

### Process Data For XGBoost

In [14]:
print( "\nProcessing data for XGBoost ...")

x_train,y_train,x_test,parcelid=loader(prop_lightgbm_path,['parcelid', 'logerror', 'transactiondate'],outlier_bound=clip_norm,labelencode=True)
features=category_features+numerical_features+extra_features
y_mean = np.mean(y_train)
dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)


Processing data for XGBoost ...

Reading data from disk ...
   Read properties file ...
   ...
   Label Encoding ...
   ...
   Read training file ...
   ...
((102914, 57), (102914L,))

Prepare for prediction ...
   Read sample file ...
   ...
   Merge with property data ...
(54225, 57)


### Set XGBoost Hyper Parameters

In [15]:
xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'base_score': y_mean,
    'silent': 1
}
num_boost_rounds = 250

### Run XGBoost

In [16]:
print("\nFitting XGBoost model ...")

model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)


Fitting XGBoost model ...


### XGBoost Prediction

In [17]:
print( "\nPredicting with XGBoost ...")

xgb_pred1 = model.predict(dtest)

print( "\nFirst XGBoost predictions:" )
print( pd.DataFrame(xgb_pred1).head() )


Predicting with XGBoost ...

First XGBoost predictions:
          0
0  0.003354
1  0.008021
2 -0.002644
3  0.009713
4 -0.001039


### Validate For First XGBoost

In [18]:
validate(np.clip(np.tile(xgb_pred1[:,None],[1,6]),clip_norm[0],clip_norm[1]))

0.06695654820228315

### Set XGBoost Hyper Parameters Again

In [19]:
xgb_params = {
    'eta': 0.033,
    'max_depth': 6,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}
num_boost_rounds = 150

### Run XGBoost Again

In [20]:
print( "\nTraining XGBoost again ...")

model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)


Training XGBoost again ...


### XGBoost Prediction Again

In [21]:
print( "\nPredicting with XGBoost again ...")

xgb_pred2 = model.predict(dtest)

print( "\nSecond XGBoost predictions:" )
print( pd.DataFrame(xgb_pred2).head() )


Predicting with XGBoost again ...

Second XGBoost predictions:
          0
0  0.003513
1  0.008814
2 -0.000905
3  0.009317
4  0.000328


### Validate For Second XGBoost

In [22]:
validate(np.clip(np.tile(xgb_pred2[:,None],[1,6]),clip_norm[0],clip_norm[1]))

0.0669726678132372

### Combine Twice XGBoost Predictions

In [23]:
xgb_pred = XGB1_WEIGHT*xgb_pred1 + (1-XGB1_WEIGHT)*xgb_pred2

print( "\nCombined XGBoost predictions:" )
print( pd.DataFrame(xgb_pred).head() )


Combined XGBoost predictions:
          0
0  0.003386
1  0.008179
2 -0.002297
3  0.009634
4 -0.000765


### Validate For Combined XGBoost

In [24]:
validate(np.clip(np.tile(xgb_pred[:,None],[1,6]),clip_norm[0],clip_norm[1]))

0.06694858812380969

### Clean for XGBoost

In [25]:
del x_train,y_train,x_test,parcelid,dtrain,dtest
gc.collect()

199

6. Neural Network
----------------------

### Process Data For Neural Network

In [26]:
print( "\n\nProcessing data for Neural Network ...")

x_train,y_train,x_test,parcelid=loader(prop_lightgbm_path,['parcelid', 'logerror','transactiondate']+extra_features,parse_date=True,onehotencode=True)
ssr=normalize(x_test)
x_train=ssr.transform(x_train)

print(x_train.shape)
len_x=x_train.shape[1]



Processing data for Neural Network ...

Reading data from disk ...
   Read properties file ...
   ...
   One Hot Encoding ...
   ...
   Read training file ...
   ...
((105121, 177), (105121L,))

Prepare for prediction ...
   Read sample file ...
   ...
   Merge with property data ...
(54225, 177)
(105121L, 177L)


### Set Neural Network

In [27]:
print("\nSetting up neural network model...")
nn = Sequential()
nn.add(Dense(units = 400 , kernel_initializer = 'normal', input_dim = len_x))
nn.add(PReLU())
nn.add(Dropout(.4))
nn.add(Dense(units = 160 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(units = 64 , kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.5))
nn.add(Dense(units = 26, kernel_initializer = 'normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(1, kernel_initializer='normal'))
nn.compile(loss='mae', optimizer=Adam(lr=4e-3, decay=1e-4))


Setting up neural network model...


### Run Neural Network

In [28]:
print("\nFitting neural network model...")

nnhistory=nn.fit(np.array(x_train), np.array(y_train), batch_size = 4096, epochs = 70, verbose=0)

print(nnhistory.history['loss'][-1])


Fitting neural network model...
0.0699696842884


### Neural Network Prediction

In [29]:
print("\nPredicting with neural network model...")

preds_dictionary=OrderedDict()
preds_dictionary['parcelid']=parcelid
for i,date in enumerate(prediction_columns):
    x_test_i=x_test.copy()
    x_test_i["transactiondate_year"] = date[:4]
    x_test_i["transactiondate_month"] = date[4:]
    x_test_i=ssr.transform(x_test_i)
    preds_dictionary[date]=nn.predict(x_test_i).flatten()
    print "   ..."
    del x_test_i
nn_pred = pd.DataFrame(preds_dictionary)

print( "\nNeural Network predictions:" )
print( pd.DataFrame(nn_pred).head() )


Predicting with neural network model...
   ...
   ...
   ...
   ...
   ...
   ...

Neural Network predictions:
      parcelid    201607    201608    201609    201707    201708    201709
782   14677191  0.004404  0.004402  0.004400  0.004405  0.004403  0.004401
968   11183209  0.004399  0.004403  0.004407  0.004400  0.004403  0.004407
1165  11554091  0.004373  0.004375  0.004378  0.004376  0.004377  0.004379
1351  11742566  0.004386  0.004388  0.004393  0.004387  0.004389  0.004394
1609  14667297  0.004405  0.004404  0.004403  0.004406  0.004405  0.004404


### Validate For NN

In [30]:
validate(np.clip(nn_pred.values[:,1:],clip_norm[0],clip_norm[1]))

0.06760387642306702

### Clean for NN

In [31]:
del x_train,y_train,x_test,parcelid,preds_dictionary
gc.collect()

4583

7. OLS
-------------------

### Process Data For OLS

In [32]:
print( "\n\nProcessing data for OLS ...")
x_train,y_train,x_test,parcelid=loader(prop_lightgbm_path,['parcelid', 'logerror','transactiondate']+extra_features,parse_date=True,onehotencode=True)
ssr=normalize(x_test)
x_train=ssr.transform(x_train)

print(x_train.shape)



Processing data for OLS ...

Reading data from disk ...
   Read properties file ...
   ...
   One Hot Encoding ...
   ...
   Read training file ...
   ...
((105121, 177), (105121L,))

Prepare for prediction ...
   Read sample file ...
   ...
   Merge with property data ...
(54225, 177)
(105121L, 177L)


### Set OLS

In [33]:
np.random.seed(17)
random.seed(17)

### Run OLS

In [34]:
print("\nFitting OLS...")

reg = LinearRegression(n_jobs=-1)
reg.fit(x_train, y_train)


Fitting OLS...


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

### OLS Prediction

In [35]:
print("\nPredicting with OLS model...")

preds_dictionary=OrderedDict()
preds_dictionary['parcelid']=parcelid
for i,date in enumerate(prediction_columns):
    x_test_i=x_test.copy()
    x_test_i["transactiondate_year"] = date[:4]
    x_test_i["transactiondate_month"] = date[4:]
    x_test_i=ssr.transform(x_test_i)
    preds_dictionary[date]=reg.predict(x_test_i)
    print "   ..."
    del x_test_i
ols_pred = pd.DataFrame(preds_dictionary)

print( "\nOLS predictions:" )
print( pd.DataFrame(ols_pred).head() )


Predicting with OLS model...
   ...
   ...
   ...
   ...
   ...
   ...

OLS predictions:
      parcelid    201607    201608    201609    201707    201708    201709
782   14677191 -0.006553 -0.008514 -0.010476 -0.002206 -0.004168 -0.006130
968   11183209  0.013480  0.011519  0.009557  0.017826  0.015865  0.013903
1165  11554091 -0.005324 -0.007286 -0.009248 -0.000978 -0.002940 -0.004901
1351  11742566  0.023170  0.021208  0.019246  0.027516  0.025554  0.023593
1609  14667297 -0.017631 -0.019592 -0.021554 -0.013284 -0.015246 -0.017208


### Validate For OLS

In [36]:
validate(np.clip(ols_pred.values[:,1:],clip_norm[0],clip_norm[1]))

0.0680084796120211

### Clean for OLS

In [37]:
del x_train,y_train,x_test,preds_dictionary
gc.collect()

232

8. Combine And Save
----------------------------------

### Combine Predictionsm

In [38]:
print( "\nCombining XGBoost, LightGBM, and baseline predicitons ..." )
lgb_weight = 1 - XGB_WEIGHT - BASELINE_WEIGHT - NN_WEIGHT - OLS_WEIGHT 
pred0 = BASELINE_WEIGHT*BASELINE_PRED
pred0 += lgb_weight*lgb_pred
pred0 += XGB_WEIGHT*xgb_pred

print( "\nCombined XGB/LGB/baseline predictions:" )
print( pd.DataFrame(pred0).head() )

print( "\nCombining with XGB/LGB/NN/OLS/baseline predicitons: ..." )
pred0 =np.tile(pred0[:,None],[1,6]) + NN_WEIGHT*nn_pred.values[:,1:]
pred0 = FUDGE_FACTOR * ( OLS_WEIGHT*ols_pred.values[:,1:] + pred0 )
submission_dict=OrderedDict()
submission_dict['parcelid']=parcelid
for i,date in enumerate(prediction_columns):
    submission_dict[date]=pred0[:,i]
submission=pd.DataFrame(submission_dict)
for c in submission.columns[1:]:
    submission[c]=submission[c].apply(lambda x:float(format(x, '.4f')))

print( "\nCombined XGB/LGB/NN/baseline/OLS predictions:" )
print( submission.head() )


Combining XGBoost, LightGBM, and baseline predicitons ...

Combined XGB/LGB/baseline predictions:
          0
0  0.007198
1  0.010301
2  0.003876
3  0.010879
4  0.004672

Combining with XGB/LGB/NN/OLS/baseline predicitons: ...

Combined XGB/LGB/NN/baseline/OLS predictions:
      parcelid  201607  201608  201609  201707  201708  201709
782   14677191  0.0080  0.0079  0.0077  0.0083  0.0082  0.0080
968   11183209  0.0129  0.0127  0.0126  0.0132  0.0130  0.0129
1165  11554091  0.0044  0.0042  0.0041  0.0047  0.0045  0.0044
1351  11742566  0.0142  0.0141  0.0139  0.0145  0.0144  0.0142
1609  14667297  0.0044  0.0043  0.0041  0.0047  0.0046  0.0044


In [39]:
validate(np.clip(submission.values[:,1:],clip_norm[0],clip_norm[1]))

0.06703410937545362